In [ ]:
#Library import
import pyodbc, os, string, sys, datetime, calendar, time, shutil, traceback
from ConfigParser import SafeConfigParser
from datetime import timedelta
from datetime import date
from time import sleep

#create methods to connect and disconnect from Teradata
def create_connection (box):
	conn_string = get_conn_string(box)
	cnxn = pyodbc.connect(conn_string)
	return cnxn.cursor(), cnxn

def kill_connection (cursor, cnxn):
	cursor.close()
	cnxn.close()

#Environment variable
def get_conn_string(env_str):
	username = "[your username]"
	password = "[your password]"
	if env_str == "mz":
		return "DSN=mozart;Database=ACCESS_VIEWS;UID=" + username + ";PWD=" + password + ";"
	if env_str == "hp":
		return "DSN=hopper;Database=ACCESS_VIEWS;UID=" + username + ";PWD=" + password + ";"

#Initialize run date
start_dt=datetime.date(2017,2,5)
end_dt=datetime.date(2017,2,11)
run_dt=start_dt

cursor, cnxn = create_connection("hp")  # change connection string as required

while run_dt<=end_dt:
    # SQL script start here:
    str_sql_1 = """
    del p_csi_tbs_t.bh_srch_sessions_srt;
    """

    str_sql_2 = """
    insert into p_csi_tbs_t.bh_srch_sessions_srt (session_start_dt, guid, session_skey, site_id, cobrand, device_type_level1, experience_level2, cust_traffic_source_level2, user_id, first_search_event_dtl)
    select
    e.session_start_dt
    ,e.guid
    ,e.session_skey
    ,e.site_id
    ,s.cobrand
    ,s.device_type_level1
    ,s.experience_level2
	,s.cust_traffic_source_level2
	,COALESCE(s.parent_uid, s.signedin_user_id, s.mapped_user_id, 0) AS user_id
	,MIN('event_ts=' || CAST(event_timestamp AS VARCHAR(19)) || '&first_search_seqnum=' || CAST(seqnum AS VARCHAR(10))) AS first_search_event_dtl
	from ubi_v.ubi_event e
    inner join p_soj_cl_v.clav_session_ext AS s ON (e.guid=s.guid AND e.session_skey=s.session_skey AND e.session_start_dt=s.session_start_dt AND e.site_id=s.site_id)

    where
    e.session_start_dt = '""" + str(run_dt) + """'
    and s.session_start_dt = '""" + str(run_dt) + """'
    and s.cobrand IN (6)
    and e.site_id IN (0,3,77)
    and s.site_id IN (0,3,77)
    and (e.page_id IN (1677950,1468757,2046791,1468757,2051457,2052261,2059706,2052261,2053742,2054032,2047936,5387) OR (e.page_id = 2351460 AND sojlib.soj_nvl(e.soj, 'eactn') = 'EXPC'))
    and s.exclude = 0
    and e.rdt = 0
    and s.valid_page_count > 1
    and e.sqr IS NOT NULL
    and sojlib.soj_extract_flag(e.flags, 267) <> 1
    and (sojlib.soj_nvl(e.soj, 'gf') IS NULL OR POSITION('LH_Sold' IN sojlib.soj_nvl(e.soj, 'gf') ) = 0)
    and e.url_query_string NOT LIKE '%LH_Sold=1%'
    and e.url_query_string NOT LIKE '%LH_Complete=1%'
    group by 1,2,3,4,5,6,7,8,9;
    """

    str_sql_3 = """
    insert into p_csi_tbs_t.bh_srch_sessions_srt (session_start_dt, guid, session_skey, site_id, cobrand, device_type_level1, experience_level2, cust_traffic_source_level2, user_id, first_search_event_dtl)
    select
    e.session_start_dt
    ,e.guid
    ,e.session_skey
    ,e.site_id
    ,s.cobrand
    ,s.device_type_level1
    ,s.experience_level2
    ,s.cust_traffic_source_level2
    ,COALESCE(s.parent_uid, s.signedin_user_id, s.mapped_user_id, 0) AS user_id
    ,MIN('event_ts=' || CAST(event_timestamp AS VARCHAR(19)) || '&first_search_seqnum=' || CAST(seqnum AS VARCHAR(10))) AS first_search_event_dtl
    from ubi_v.ubi_event e inner join p_soj_cl_v.clav_session_ext AS s ON (e.guid=s.guid AND e.session_skey=s.session_skey AND e.session_start_dt=s.session_start_dt AND e.site_id=s.site_id)
    where
    e.session_start_dt = '""" + str(run_dt) + """'
    and s.session_start_dt = '""" + str(run_dt) + """'
    and s.cobrand IN (0,7)
    and e.site_id IN (0,3,77)
    and s.site_id IN (0,3,77)
    and e.page_id IN (2045573,3286,1637,4999)
    and s.exclude = 0
    and e.rdt = 0
    and s.valid_page_count > 1
    and e.sqr IS NOT NULL
    and sojlib.soj_extract_flag(e.flags, 267) <> 1
    and (sojlib.soj_nvl(e.soj, 'gf') IS NULL OR POSITION('LH_Sold' IN sojlib.soj_nvl(e.soj, 'gf') ) = 0)
    and e.url_query_string NOT LIKE '%LH_Sold=1%'
    and e.url_query_string NOT LIKE '%LH_Complete=1%'
    group by 1,2,3,4,5,6,7,8,9;
    """

    str_sql_4 = """
    update p_csi_tbs_t.bh_srch_sessions_srt
    set first_srch_seqnum=cast(sojlib.soj_nvl(first_search_event_dtl,'first_search_seqnum') as integer)
    where p_csi_tbs_t.bh_srch_sessions_srt.session_start_dt= '""" + str(run_dt) + """'
    and p_csi_tbs_t.bh_srch_sessions_srt.first_srch_seqnum is null;
    """

    str_sql_5 = """
    collect stats on P_CSI_TBS_T.bh_srch_sessions_srt INDEX ( guid );
    """

    str_sql_6 = """
    del p_csi_tbs_t.bh_srch_sess_txn_dtl_srt;
    """

    str_sql_7 = """
    insert into p_csi_tbs_t.bh_srch_sess_txn_dtl_srt (session_start_dt, guid, session_skey, site_id, cobrand, device_type_level1, experience_level2, cust_traffic_source_level2, user_id, first_srch_seqnum, created_dt, auct_end_dt, item_id, transaction_id, item_price, quantity, lstg_curncy_exchng_rate, lstg_curncy_id, EXCL_PRIOR_SRCH_SORT, core_yn)
    select
    s.session_start_dt
    ,s.guid
    ,s.session_skey
    ,s.site_id
    ,s.cobrand
    ,s.device_type_level1
    ,s.experience_level2
    ,s.cust_traffic_source_level2
    ,s.user_id
    ,s.first_srch_seqnum
    ,co.created_dt
    ,co.auct_end_dt
    ,co.item_id
    ,co.transaction_id
    ,co.item_price
    ,co.quantity
    ,co.lstg_curncy_exchng_rate
    ,co.lstg_curncy_id
    ,CASE WHEN p.EXCL_PRIOR_SRCH_SORT IS NULL THEN 'Unknown'
    				WHEN p.EXCL_PRIOR_SRCH_SORT = 'BM' THEN 'Best Match'
    				WHEN p.EXCL_PRIOR_SRCH_SORT IN ('Error', 'Other') THEN 'Unknown'
    				ELSE 'Deterministic'
    END AS EXCL_PRIOR_SRCH_SORT
    ,CASE
    	WHEN cat.sap_category_id NOT IN (5,7,23,41) THEN 1
    	WHEN cat.sap_category_id IS NOT NULL THEN 0
    	END AS core_yn

    from p_csi_tbs_t.bh_srch_sessions_srt s

    LEFT JOIN p_soj_cl_v.checkout_metric_item co ON (s.guid = co.bbowa_guid AND s.session_skey = co.bbowa_session_skey AND s.session_start_dt = co.bbowa_session_start_dt AND s.site_id = co.bbowa_site_id AND s.cobrand = co.bbowa_cobrand AND s.first_srch_seqnum < co.bbowa_seqnum AND co.created_dt BETWEEN '""" + str(run_dt) + """'  AND (CAST(('""" + str(run_dt) + """') AS DATE)+13) AND co.auct_end_dt >= '""" + str(run_dt) + """' AND co.ck_wacko_yn = 'N')
    LEFT JOIN EXL_PPS_PURCHASES AS p

    on  p.item_id=co.item_id
    and p.SUCC_BID_TXN_ID=co.TRANSACTION_ID
    and p.SESSION_START_DT=co.sess_SESSION_START_DT
    and p.guid=co.sess_GUID
    and p.session_skey=co.sess_session_skey
    and p.soj_site_id=co.sess_site_id
    and p.auct_type_code IN (1,2,7,8,9)
    and p.SEQNUM>=s.first_srch_seqnum

    LEFT JOIN access_views.dw_category_groupings cat ON (co.item_site_id = cat.site_id AND co.leaf_categ_id = cat.leaf_categ_id)

    WHERE
    s.session_start_dt = '""" + str(run_dt) + """'
    ;
    """

    str_sql_8 = """
    collect statistics on p_csi_tbs_t.bh_srch_sess_txn_dtl_srt INDEX ( guid );
    """

    str_sql_9 = """
    del p_csi_tbs_t.bh_srch_sess_txn_srt;
    """

    str_sql_10 = """
    insert into p_csi_tbs_t.bh_srch_sess_txn_srt (session_start_dt, guid, session_skey, site_id, cobrand, device_type_level1, experience_level2, cust_traffic_source_level2, css_session_core, tot_srch_sessions, purchases_core, bi_qty_core, gmb_usd_pr_core,css_session_core_BM, css_session_core_DET, css_session_core_UNK)
    select
    s.session_start_dt
    ,s.guid
    ,s.session_skey
    ,s.site_id
    ,s.cobrand
    ,s.device_type_level1
    ,s.experience_level2
    ,s.cust_traffic_source_level2
    ,COUNT(DISTINCT CASE WHEN s.core_yn = 1 AND s.item_id > 0 THEN s.guid||s.session_skey END) AS css_session_core
    ,COUNT(DISTINCT s.guid||s.session_skey) AS tot_srch_sessions
    ,COUNT(DISTINCT CASE WHEN s.core_yn = 1 AND s.item_id > 0 THEN s.transaction_id END) AS purchases_core
    ,SUM(s.quantity) AS bi_qty_core
    ,SUM(CASE WHEN s.core_yn = 1 THEN CAST(s.item_price * s.quantity * pr.curncy_plan_rate AS DECIMAL(18,2)) ELSE 0 END) AS gmb_usd_pr_core

    ,SUM(CASE WHEN s.core_yn = 1 AND s.EXCL_PRIOR_SRCH_SORT = 'Best Match' THEN 1 ELSE 0 END) AS css_session_core_BM
    ,SUM(CASE WHEN s.core_yn = 1 AND s.EXCL_PRIOR_SRCH_SORT = 'Deterministic' THEN 1 ELSE 0 END) AS css_session_core_DET
    ,SUM(CASE WHEN s.core_yn = 1 AND s.EXCL_PRIOR_SRCH_SORT = 'Unknown' THEN 1 ELSE 0 END) AS css_session_core_UNK

    from p_csi_tbs_t.bh_srch_sess_txn_dtl_srt s
    LEFT JOIN access_views.ssa_curncy_plan_rate_dim pr ON (s.lstg_curncy_id = pr.curncy_id)

    where s.session_start_dt = '""" + str(run_dt) + """'

    group by 1,2,3,4,5,6,7,8;
	"""

    str_sql_11 = """
    collect statistics on P_CSI_TBS_T.bh_srch_sess_txn_srt index (session_start_dt ,site_id ,experience_level2 ,cust_traffic_source_level2);
    """

    str_sql_12 = """
    collect statistics on P_CSI_TBS_T.bh_srch_sess_txn_srt column partition;
    """

    str_sql_13 = """
    insert into p_csi_tbs_t.bh_gss_srt
    select
    session_start_dt
    ,site_id
    ,cobrand
    ,device_type_level1
    ,experience_level2
    ,cust_traffic_source_level2
    ,SUM(css_session_core)
    ,SUM(tot_srch_sessions)
    ,SUM(purchases_core)
    ,SUM(bi_qty_core)
    ,SUM(gmb_usd_pr_core)

    ,SUM( CASE WHEN css_session_core_BM > 0 AND css_session_core_DET = 0 THEN 1 ELSE 0 END) AS CSS_BM
    ,SUM( CASE WHEN css_session_core_BM = 0 AND css_session_core_DET > 0 THEN 1 ELSE 0 END) AS CSS_DET
    ,SUM( CASE WHEN css_session_core_BM > 0  AND css_session_core_DET > 0 THEN 1 ELSE 0 END) AS CSS_BM_DET
    ,SUM( CASE WHEN css_session_core_UNK>0 and css_session_core_BM=0 and css_session_core_DET=0 THEN 1 ELSE 0 END) AS CSS_UNK

    ,sum(case when css_session_core_BM>0 and css_session_core_DET=0 then gmb_usd_pr_core else 0 end) as gmb_usd_pr_core_BM
    ,sum(case when css_session_core_BM=0 and css_session_core_DET>0 then gmb_usd_pr_core else 0 end) as gmb_usd_pr_core_DET
    ,sum(case when css_session_core_BM>0 and css_session_core_DET>0 then gmb_usd_pr_core else 0 end) as gmb_usd_pr_core_BM_DET
    ,sum(case when css_session_core_UNK>0 and css_session_core_BM=0 and css_session_core_DET=0 then gmb_usd_pr_core else 0 end) as gmb_usd_pr_core_UNK

    from p_csi_tbs_t.bh_srch_sess_txn_srt
    where session_start_dt = '""" + str(run_dt) + """'
    group by 1,2,3,4,5,6;
    """

    str_sql_14 = """
    collect stats p_csi_tbs_t.bh_gss_srt index(session_start_dt, site_id, experience_level2, cust_traffic_source_level2);
    """

    cursor.execute(str_sql_1)
    cnxn.commit()
    cursor.execute(str_sql_2)
    cnxn.commit()
    cursor.execute(str_sql_3)
    cnxn.commit()
    cursor.execute(str_sql_4)
    cnxn.commit()
    cursor.execute(str_sql_5)
    cnxn.commit()
    cursor.execute(str_sql_6)
    cnxn.commit()
    cursor.execute(str_sql_7)
    cnxn.commit()
    cursor.execute(str_sql_8)
    cnxn.commit()
    cursor.execute(str_sql_9)
    cnxn.commit()
    cursor.execute(str_sql_10)
    cnxn.commit()
    cursor.execute(str_sql_11)
    cnxn.commit()
    cursor.execute(str_sql_12)
    cnxn.commit()
    cursor.execute(str_sql_13)
    cnxn.commit()
    cursor.execute(str_sql_14)
    cnxn.commit()

    print(run_dt)

    run_dt = run_dt + datetime.timedelta(1)  # increment by 1 day for now

#Now we kill the connections
kill_connection(cursor, cnxn)
